In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [9]:
df_alltransactions = pd.read_csv('Data/1_alltransactions.csv')

In [10]:
df_alltransactions.head()

,Unnamed: 0,S/N,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale
0,0,1,LANDED HOUSING DEVELOPMENT,JALAN PARAS,Terrace,14.0,OCR,Freehold,Resale,1.0,2150000.0,-,1888.0,Land,-,1139.0,Mar-2018
1,1,2,FRANKEL ESTATE,COLDSTREAM AVENUE,Detached,15.0,OCR,Freehold,Resale,1.0,10500000.0,-,9093.0,Land,-,1155.0,Mar-2018
2,2,3,LANDED HOUSING DEVELOPMENT,WOO MON CHEW ROAD,Semi-detached,15.0,OCR,Freehold,Resale,1.0,3330000.0,-,2472.0,Land,-,1347.0,Mar-2018
3,3,4,LANDED HOUSING DEVELOPMENT,JALAN SUKACHITA,Terrace,13.0,OCR,Freehold,Resale,1.0,2288000.0,-,1647.0,Land,-,1389.0,Mar-2018
4,4,5,LANDED HOUSING DEVELOPMENT,EAST COAST ROAD,Semi-detached,15.0,OCR,Freehold,Resale,1.0,4550000.0,-,3314.0,Land,-,1373.0,Mar-2018
